# Q1. Running Elastic

In [1]:
!curl localhost:9200

{
  "name" : "a47f12568074",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "Vk-UIgGMS9COHd7ozittxA",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


The version build hash 42f05b9372a9a4a470db3b52817899b99a76ee73

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

# Q2. Indexing the data

In [3]:
from elasticsearch import Elasticsearch

In [4]:
es_client = Elasticsearch('http://localhost:9200')

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

In [6]:
index_name = "course-questions"

In [7]:
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [8]:
for doc in documents:
    es_client.index(index=index_name, document=doc)

We use the index method to add data to elastic.

# Q3. Searching

In [9]:
query = "How do I execute a command in a running docker container?"

In [11]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

In [12]:
resp = es_client.search(index=index_name, body=search_query)

In [18]:
for hit in resp['hits']['hits']:
    print(hit['_score'])

84.050095
75.54128
72.08518
51.04628
49.938507


The score of the top ranking result is 84.05

# Q4. Filtering

In [26]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [27]:
resp = es_client.search(index=index_name, body=search_query)

In [28]:
for hit in resp['hits']['hits']:
    print(hit['_source']['question'])
    print()

How do I debug a docker container?

How do I copy files from my local machine to docker container?

How do I copy files from a different folder into docker container’s working directory?



The third question is : "How do I copy files from a different folder into docker container’s working directory?"

# Q5. Building a prompt

In [54]:
context_template = """
Q: {question}
A: {text}
""".strip()

context = []

for hit in resp['hits']['hits']:
    question = hit['_source']['question']
    text = hit['_source']['text']
    context.append(context_template.format(question=question, text=text))

context = "\n\n".join(context)

In [55]:
context

'Q: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani\n\nQ: How do I copy files from a different folder into docker container’s working directory?\nA: You can copy files from your local machine into a Docker container using the docker cp comman

In [56]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [57]:
question = "How do I execute a command in a running docker container?"

In [58]:
prompt = prompt_template.format(question=question, context=context)

In [59]:
prompt

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: How do I execute a command in a running docker container?\n\nCONTEXT:\nQ: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_dir

In [60]:
len(prompt)

1462

The length of the resulting prompt is 1462.

# Q6. Tokens

In [61]:
import tiktoken

In [62]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [68]:
tokens = encoding.encode(prompt)

In [69]:
len(tokens)

322

In [70]:
for token in tokens:
    print(encoding.decode_single_token_bytes(token))    

b"You're"
b' a'
b' course'
b' teaching'
b' assistant'
b'.'
b' Answer'
b' the'
b' QUESTION'
b' based'
b' on'
b' the'
b' CONT'
b'EXT'
b' from'
b' the'
b' FAQ'
b' database'
b'.\n'
b'Use'
b' only'
b' the'
b' facts'
b' from'
b' the'
b' CONT'
b'EXT'
b' when'
b' answering'
b' the'
b' QUESTION'
b'.\n\n'
b'QUESTION'
b':'
b' How'
b' do'
b' I'
b' execute'
b' a'
b' command'
b' in'
b' a'
b' running'
b' docker'
b' container'
b'?\n\n'
b'CON'
b'TEXT'
b':\n'
b'Q'
b':'
b' How'
b' do'
b' I'
b' debug'
b' a'
b' docker'
b' container'
b'?\n'
b'A'
b':'
b' Launch'
b' the'
b' container'
b' image'
b' in'
b' interactive'
b' mode'
b' and'
b' overriding'
b' the'
b' entry'
b'point'
b','
b' so'
b' that'
b' it'
b' starts'
b' a'
b' bash'
b' command'
b'.\n'
b'docker'
b' run'
b' -'
b'it'
b' --'
b'entry'
b'point'
b' bash'
b' <'
b'image'
b'>\n'
b'If'
b' the'
b' container'
b' is'
b' already'
b' running'
b','
b' execute'
b' a'
b' command'
b' in'
b' the'
b' specific'
b' container'
b':\n'
b'docker'
b' ps'
b' ('
b'find'
b' the'

The prompt has 322 tokens